# Set up

In [ ]:
!git clone https://github.com/NLP-Reichman/2025_assignment_1.git
!mv 2025_assignment_1/data data
!rm 2025_assignment_1/ -r

Cloning into '2025_assignment_1'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 62 (delta 18), reused 44 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (62/62), 7.05 MiB | 5.67 MiB/s, done.
Resolving deltas: 100% (18/18), done.


### Additional functions (move to separate location?)

In [12]:
import os
import json
from typing import Dict, List, Tuple
import copy
def load_data_files() -> Dict[str, str]:
    files = {}
    for file in os.listdir('data'):
        if '.json' not in file:
            continue
        texts = json.load(open(f'data/{file}', encoding='utf-8'))['tweet_text']
        files[file.replace('.json', '')] = [val for _, val in texts.items()]
    return files

def probs_for_counts(c_vect : Dict, full_copy = True, add_unknown = False) -> Dict[List, Dict[str, float]]:
    if full_copy:
        prob_vect = copy.deepcopy(c_vect)
    else:
        prob_vect = c_vect # modifying a dict in place is not a good idea
    for vocab_key, vocab_val in c_vect.items():
        s = sum(vocab_val.values())
        if add_unknown:
            un_s = s/(len(vocab_val)-1)
            prob_vect[vocab_key]["<unk>"] = un_s
            s += un_s
        for k, v in vocab_val.items():
            prob_vect[vocab_key][k] = v/s
    return prob_vect

def get_base_model_table(texts, vocab_size) -> Tuple[Dict[List, Dict[str, int]], Dict[List, Dict[str, float]]]:
    internal_vect = {}
    starting_message = (vocab_size-1)*["<start>"]
    for text in texts:
        current_message = starting_message
        full_text = list(text) + ["<end>"]
        for token in full_text:
            conv = tuple(current_message)
            internal_vect.setdefault(conv, {})
            internal_vect[conv].setdefault(token, 0)
            internal_vect[conv][token] += 1
            if vocab_size > 1:
                current_message = current_message[1:] + [token]
    prob_vect = probs_for_counts(internal_vect)
    return internal_vect, prob_vect

# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [13]:
import os
import json
import pandas as pd
import numpy as np
from itertools import product
import math

# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data. The vocabulary should include the `<start>` and  `<end>` tokens.

Note - do NOT lowecase the sentences.

In [14]:
def preprocess() -> list[str]:
  '''
  Return a list of characters, representing the shared vocabulary of all languages
  '''
  vocab = []
  lang_files = load_data_files()
  for _, lang_data in lang_files.items():
    for message in lang_data:
      for c in message:
        vocab.append(c)
  vocab = list(set(vocab))
  vocab.append("<start>")
  vocab.append("<end>")
  return vocab

In [15]:
vocab = preprocess()
print(f"vocab length: {len(vocab)}")
print(f"Some characters in the vocab: {vocab[:10]}")

vocab length: 1804
Some characters in the vocab: ['📆', '엣', 'が', '主', '🦁', '文', '🤗', 'б', '💔', 'บ']


## Part 2
Implement the function *build_lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. To ensure consistent probabilities calculation, please add n-1 `<start>` tokens to the beginning of a tweet and one `<end>` token at the end. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add-one smoothing information to the dictionary and implement it.

Please add the `<unk>` token with $p(<unk>)=1/|V|$ to the LM if buiulding a smoothed LM.

In [16]:
import copy
import tqdm
def build_lm(lang: str, n: int, smoothed: bool = False) -> dict[str, dict[str, float]]:
  '''
  Return a language model for the given lang and n_gram (n)
  :param lang: the language of the model
  :param n: the n_gram value
  :param smoothed: boolean indicating whether to apply smoothing
  :return: a dictionary where the keys are n_grams and the values are dictionaries
  '''
  LM = {}
  language = load_data_files()[lang]
  c_vect, prob_vect = get_base_model_table(language, n)
  vocab = preprocess() + ["<unk>"]
  if not smoothed:
    LM = prob_vect
  else:
    smoothed_c_vect = copy.deepcopy(c_vect)
    for vocab_key, vocab_val in tqdm.tqdm(c_vect.items()):
        for w in vocab:
           if w not in vocab_val:
            smoothed_c_vect[vocab_key][w] = 1
    LM = probs_for_counts(smoothed_c_vect, False, True)
  LM[("<unk>",)] = {k: 1/len(vocab) for k in vocab}
  return LM

In [17]:
LM = build_lm("en", 3, False)
print(f"English Language Model with 3-gram is of length: {len(LM)}")

English Language Model with 3-gram is of length: 8239


## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

The `<unk>` should be used for unknown contexts when calculating the perplexities.

In [18]:
def perplexity(model: dict, text: list, n) -> float:
  '''
  Calculates the perplexity of the given string using the given language model.
  :param model: The language model
  :param text: The tokenized text to calculate the perplexity for
  :param n: The n-gram of the model
  :return: The perplexity
  '''
  pp = 0
  prepend = ["<start>"] * (n-1) + text + ["<end>"]
  c_text = prepend[:n-1]
  for c in prepend[n-1:]:
    conv = tuple(c_text)
    if conv not in model: # cant evaluate
      conv = ("<unk>",)
    c = c if c in model[conv] else "<unk>"
    pp += math.log2(model[conv][c])
    c_text = c_text[1:] + [c]
  return pp*(-1/len(prepend[n-1:]))

In [19]:
def eval(model: dict, target_lang: str, n: int) -> float:
  '''
  Return the perplexity value calculated over applying the model on the text file
  of the target_lang language.
  :param model: the language model
  :param target_lang: the target language
  :param n: The n-gram of the model
  :return: the perplexity value
  '''
  pp = 0
  lang = load_data_files()[target_lang]
  total_tokens = 0
  for text in lang:
    total_tokens += len(text)+1
    pp += perplexity(model, list(text), n) * (-len(text)-1)
  return 2**(pp*(-1/total_tokens))


In [20]:
LM = build_lm("en", 3, True)

100%|██████████| 8238/8238 [00:00<00:00, 8855.37it/s]


In [21]:
print("Perplexity of the English 3-gram model on datasets:")
print(f"On English: {eval(LM, 'en', 3): .2f}")
print(f"On French: {eval(LM, 'fr', 3): .2f}")
print(f"On Dutch: {eval(LM, 'nl', 3): .2f}")
print(f"On Tagalog: {eval(LM, 'tl', 3): .2f}")

Perplexity of the English 3-gram model on datasets:
On English:  28.49
On French:  61.18
On Dutch:  65.31
On Tagalog:  75.53


In [22]:
lm1 = build_lm("en", 1, True)
lm2 = build_lm("en", 2, True)
lm3 = build_lm("en", 3, True)
lm4 = build_lm("en", 4, True)

print("Perplexity on differnet n-gram models on English")
print(f"On 1-gram: {eval(lm1, 'en', 1): .2f}")
print(f"On 2-gram: {eval(lm2, 'en', 2): .2f}")
print(f"On 3-gram: {eval(lm3, 'en', 3): .2f}")
print(f"On 4-gram: {eval(lm4, 'en', 4): .2f}")

100%|██████████| 78911/78911 [00:09<00:00, 8348.73it/s]


Perplexity on differnet n-gram models on English
On 1-gram:  1755.02
On 2-gram:  21.38
On 3-gram:  28.49
On 4-gram:  69.51


## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [23]:
languages = ['en', 'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']

In [ ]:
def match() -> pd.DataFrame:
  '''
  Return a DataFrame containing one line per every language pair and n_gram.
  Each line will contain the perplexity calculated when applying the language model
  of the source language on the text of the target language.
  :return: a DataFrame containing the perplexity values
  '''
  r = []
  for n in range(1, 5):
    for source_language in languages:
      lm = build_lm(source_language, n, True)
      for target_language in languages:
        per = eval(lm, target_language, n)
        r.append({'source_lang': source_language, 'target_lang': target_language, 'n': n, 'perplexity': per})
  return pd.DataFrame.from_records(r)

## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token. For this exercise, you may assume a valid input in the respective language.

Note - The generation of the next token should be from the LM's distribution with NO smoothing.

In [25]:
import random
def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  text = ""
  lm = build_lm(lang, n, False)
  random.seed(r)
  prompt_padded = ["<start>"] * n + list(prompt)
  generated = []
  current_prompt = prompt_padded[-(n-1):] if n > 1 else []
  while len(generated) < number_of_tokens:
    rn = random.choices(list(lm[tuple(current_prompt)].keys()), np.array(list(lm[tuple(current_prompt)].values())), k=1)[0]
    if not (rn == "<start>" or rn == "<end>" or rn == "<unk>"): # assume that number of tokens overriders <end> and not only limits sentence size
        generated.append(rn)
        if n > 1:
            current_prompt = current_prompt[1:] + [rn] 
  return prompt + "".join(generated)

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

In [26]:
print(generate('en', 1, "I am", 10, 5))
print(generate('en', 2, "I am", 10, 5))
print(generate('en', 3, "I am", 10, 5))
print(generate('en', 4, "I am ", 10, 5))
print(generate('es', 2, "Soy", 10, 5))
print(generate('es', 3, "Soy", 10, 5))
print(generate('fr', 2, "Je suis", 10, 5))
print(generate('fr', 3, "Je suis", 10, 5))

I amtpgLpC eLh
I amoulpeginSh
I amit: Lynmkm
I am carrive Ob
Soycalíodenye
Soy orbershag
Je suis:/opapropa
Je suis tunes #AC


# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [27]:
########################################
# PLACE TESTS HERE #
# Create tests
def test_preprocess():
    return {
        'vocab_length': len(preprocess()),
    }

def test_build_lm():
    return {
        'english_2_gram_length': len(build_lm('en', 2, True)),
        'english_3_gram_length': len(build_lm('en', 3, True)),
        'french_3_gram_length': len(build_lm('fr', 3, True)),
        'spanish_3_gram_length': len(build_lm('es', 3, True)),
    }

def test_eval():
    lm = build_lm('en', 3, True)
    return {
        'en_on_en': round(eval(lm, 'en', 3), 2),
        'en_on_fr': round(eval(lm, 'fr', 3), 2),
        'en_on_tl': round(eval(lm, 'tl', 3), 2),
        'en_on_nl': round(eval(lm, 'nl', 3), 2),
    }

def test_match():
    df = match()
    return {
        'df_shape': df.shape,
        'en_en_3': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 3)]['perplexity'].values[0],
        'en_tl_3': df[(df['source'] == 'en') & (df['target'] == 'tl') & (df['n'] == 3)]['perplexity'].values[0],
        'en_nl_3': df[(df['source'] == 'en') & (df['target'] == 'nl') & (df['n'] == 3)]['perplexity'].values[0],
    }

def test_generate():
    return {
        'english_2_gram': generate('en', 2, "I am", 20, 5),
        'english_3_gram': generate('en', 3, "I am", 20, 5),
        'english_4_gram': generate('en', 4, "I Love", 20, 5),
        'spanish_2_gram': generate('es', 2, "Soy", 20, 5),
        'spanish_3_gram': generate('es', 3, "Soy", 20, 5),
        'french_2_gram': generate('fr', 2, "Je suis", 20, 5),
        'french_3_gram': generate('fr', 3, "Je suis", 20, 5),
    }

TESTS = [test_preprocess, test_build_lm, test_eval, test_match, test_generate]

# Run tests and save results
res = {}
for test in TESTS:
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})

with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)

# Download the results.json file
#files.download('results.json')
########################################

100%|██████████| 62235/62235 [00:07<00:00, 8058.57it/s]


In [28]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
!ls -l

'ls' is not recognized as an internal or external command,
operable program or batch file.
